In [1]:
import random
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn
from tqdm import tqdm # Used to display training progress bar
import numpy as np
from scipy import misc
import time

In [2]:
import numpy as np

x = [1,1,1,1,1,1,1]

print(x)





[1, 1, 1, 1, 1, 1, 1]


In [3]:
init_scale = 0.1
learning_rate = 1.0
max_grad_norm = 5
num_layers = 1
num_steps = 20
hidden_size = 200
max_epoch = 4
max_max_epoch = 13
keep_prob = 1.0
lr_decay = 0.5
batch_size = 20
n_classes=2
vocab_size = 10000

In [4]:
def relu(L):
    return tf.nn.relu(L)
def flatten(layer):
    return tf.contrib.layers.flatten(layer)

def fully_conn(x_tensor, num_outputs, name):
    # TODO: Implement Function
    with tf.variable_scope(name):
        x = tf.contrib.layers.fully_connected(x_tensor, num_outputs = num_outputs, weights_initializer = tf.random_normal_initializer(mean=0.0, stddev=0.05, seed=None, dtype=tf.float32))
        x = tf.nn.elu(x)

    return x

def output(x_tensor, num_outputs):
    # TODO: Implement Function
    x_shape = x_tensor.get_shape().as_list()
    
    weights = tf.get_variable('weight', shape= [x_shape[1], num_outputs],
                              initializer=tf.uniform_unit_scaling_initializer(factor=1.0))
    bias = tf.Variable(tf.zeros([num_outputs]))
    out = tf.add(tf.matmul(x_tensor, weights), bias)
    
    return out


# Functions for constructing layers
def dense_layer(inputs, name):
    with tf.variable_scope(name):
        f = flatten(inputs)
        out = output(f, batch_size)
    return out

def train_layer_sg(l_t, l_sg, h_n):
    grad_true = tf.gradients(l_t, h_n)
    grad_pred = tf.gradients(l_sg, h_n)
    
    sg_loss = tf.add (tf.reduce_sum(tf.abs( tf.subtract(l_t, l_sg))) , tf.reduce_sum(tf.abs( tf.subtract(grad_pred , grad_true))))
    sg_opt = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(sg_loss, var_list=sg_vars)
    return sg_opt, sg_loss

In [5]:
is_training=1

In [6]:
 # Collections of trainable variables in each block
layer_vars = [tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/lstm")]

sg_vars = [tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/sg_block_1/"),
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/sg_block_2/"),
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope="architecture/sg_block_3/")]

In [7]:
# Ops for network architecture
with tf.variable_scope("architecture"):
    # Inputs
    with tf.variable_scope("input"):
        input_data  = tf.placeholder(tf.int32, [batch_size,num_steps], name="data")
        labels = tf.placeholder(tf.int32, [batch_size, num_steps], name="labels") # Target
        learning_rate =  tf.placeholder(tf.float32, shape =[])
        #mask = tf.placeholder(tf.float32, [num_steps, batch_size], name="mask")
        #mask = tf.expand_dims(mask, -1)
        
    with tf.variable_scope("lstm"):
        #add LSTM cell and dropout nodes
        cell = tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0)
        if is_training and keep_prob < 1:
            cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    with tf.device("/gpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, hidden_size])
        inputs = tf.nn.embedding_lookup(embedding, input_data)
        #add dropout to input units
        if is_training and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

    outputs = []
    sg_output=[]
    state = initial_state
    with tf.variable_scope("RNN"):
        for time_step in range(num_steps):
            with tf.variable_scope("LSTM_CELL"):
                if time_step > 0:
                    tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                outputs.append(tf.expand_dims(cell_output, 0))
            
            # Write code for synthetic gradient
            if time_step==5:
                with tf.variable_scope("sg_block_1",reuse=False):
                    sg_flat=dense_layer(cell_output,name="sg1")
            if time_step==10:
                with tf.variable_scope("sg_block_2",reuse=False):
                    sg_flat_1=dense_layer(cell_output,name="sg1")
            if time_step==15:
                with tf.variable_scope("sg_block_3",reuse=False):
                    sg_flat_2=dense_layer(cell_output,name="sg1")
            
            
        
        output = tf.reshape(tf.concat(outputs,1), [-1, hidden_size])
        #NOW proj has shape [batch_size, size]

        softmax_w = tf.get_variable("softmax_w", [hidden_size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        logits = tf.matmul(output, softmax_w) + softmax_b
       
        pred = tf.nn.softmax(logits)
        # Ops for training
        with tf.variable_scope("train"):
            pred_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
            pred_loss_sg1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=sg_flat, labels=labels))
            pred_loss_sg2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=sg_flat_1, labels=labels))
            pred_loss_sg3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=sg_flat_2, labels=labels)) 
        
        

        # Optimizer when using backprop
        with tf.variable_scope("backprop"):
            backprop_opt = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum = 0.9).minimize(pred_loss)
        
        
        cost = cost = tf.reduce_sum(pred_loss) / batch_size
        final_state = state
        correct_prediction = tf.equal(tf.argmax(pred,1), labels)
        accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))

        # if not is_training:
        #    return


        lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),config.max_grad_norm)
        #optimizer = tf.train.GradientDescentOptimizer(lr)
        optimizer = tf.train.AdagradOptimizer(lr)
        train_op = optimizer.apply_gradients(zip(grads, tvars))

    def assign_lr(session, lr_value):
        session.run(tf.assign(lr, lr_value))

ValueError: Dimension 0 in both shapes must be equal, but are 400 and 20 for 'architecture/RNN/train/SoftmaxCrossEntropyWithLogits' (op: 'SoftmaxCrossEntropyWithLogits') with input shapes: [400,10000], [20,20].